In [50]:
import data_extraction
import pandas as pd
import numpy as np

testing = data_extraction.DataExtractor()
df_products = testing.extract_from_s3('s3://data-handling-public/products.csv')

In [5]:
df_products.head()

,product_name,product_price,weight,category,EAN,date_added,uuid,removed,product_code
index,,,,,,,,,
0,FurReal Dazzlin' Dimples My Playful Dolphin,£39.99,1.6kg,toys-and-games,7425710935115,2005-12-02,83dc0a69-f96f-4c34-bcb7-928acae19a94,Still_avaliable,R7-3126933h
1,Tiffany's World Day Out At The Park,£12.99,0.48kg,toys-and-games,487128731892,2006-01-09,712254d7-aea7-4310-aff8-8bcdd0aec7ff,Still_avaliable,C2-7287916l
2,Tiffany's World Pups Picnic Playset,£7.00,590g,toys-and-games,1945816904649,1997-03-29,b089ef6f-b628-4e37-811d-fffe0102ba64,Still_avaliable,S7-1175877v
3,Tiffany's World Wildlife Park Adventures,£12.99,540g,toys-and-games,1569790890899,2013-03-20,d55de422-8b98-47d6-9991-e4bc4c5c0cb0,Removed,D8-8421505n
4,Cosatto Cosy Dolls Pram,£30.00,1.91kg,toys-and-games,7142740213920,2007-12-23,7945b657-cb02-4cc5-96cf-f65ed0a8f235,Still_avaliable,B6-2596063a


In [3]:
df_products['weight'].value_counts()

2.9kg      30
200g       30
2.4kg      26
100g       22
250g       21
           ..
0.86kg      1
1.75kg      1
0.443kg     1
2.565kg     1
134g        1
Name: weight, Length: 483, dtype: int64

In [51]:
def convert_product_weights(df_products):
    for i, row in df_products.iterrows():
        weight = row['weight']
        if 'kg' in weight:
            weight_kg = float(weight.replace('kg', ''))
        elif 'g' in weight:
            if 'x' in weight:
                num_packets, packet_weight = weight.split(' x ')
                weight_kg = float(num_packets) * float(packet_weight) / 1000
            else:
                weight_kg = float(weight.replace('g', '')) / 1000
        elif 'ml' in weight:
            weight_kg = float(weight.replace('ml', '')) / 1000
        elif 'oz' in weight:
            weight_kg = float(weight.replace('oz','')) / 35.274
        else:
            df_products = df_products.drop(i)
            continue

        df_products.at[i, 'weight'] = weight_kg
            

    return df_products

In [52]:
convert_product_weights(df_products)

TypeError: argument of type 'float' is not iterable

In [25]:
# locating all rows where weight has NULL values and there are 3 which entire row is NULL
null_values_df = df_stores_data[(df_stores_data['country_code']=='NULL')]


0 1.6
1 0.48
2 0.59
3 0.54
4 1.91


In [39]:
# dropping the above rows from the df
df_stores_data = df_stores_data.drop(null_values_df.index)

In [53]:
df_products.to_csv('weights_adjusted.csv', index=True)